In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

#!pip install tensorflow_decision_forests

import tensorflow_decision_forests as tfdf


In [95]:
data_test = pd.read_csv('/content/sample_data/california_housing_test.csv')
data_train = pd.read_csv('/content/sample_data/california_housing_train.csv')

data_train.isna().sum()
data_train = data_train.dropna()
data_test = data_test.dropna()
data_train['median_house_value']= data_train['median_house_value'].astype(int)
data_test['median_house_value']= data_test['median_house_value'].astype(int)

In [96]:
data_train['label'] = pd.cut(x = data_train['median_house_value'], bins = [10000,150000,250000,350000,450000, 600000], labels= [1,2,3, 4, 5])
data_test['label'] = pd.cut(x = data_test['median_house_value'], bins = [10000,150000,250000,350000,450000, 600000], labels= [1,2,3, 4, 5])
data_train = data_train.drop('median_house_value', axis=1)
data_test = data_test.drop('median_house_value', axis=1)


In [97]:
rslt_df = data_train.loc[data_train['label'].isnull()]
print(rslt_df)

Empty DataFrame
Columns: [longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, label]
Index: []


In [98]:
data_train['label'] = pd.to_numeric(data_train['label'], errors='coerce')
data_test['label'] = pd.to_numeric(data_test['label'], errors='coerce')


train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(data_train, label="label")
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)


Use /tmp/tmphhreegba as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.352427. Found 17000 examples.
Training model...
Model trained in 0:00:24.016828
Compiling model...
Model compiled.


In [99]:
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(data_test, label="label")
model.evaluate(test_ds)
model.summary()


3/3 [==============================] - 1s 120ms/step - loss: 0.0000e+00
Model: "random_forest_model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (8):
	households
	housing_median_age
	latitude
	longitude
	median_income
	population
	total_bedrooms
	total_rooms

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.      "median_income"  0.464106 ################
    2.           "latitude"  0.294892 ########
    3.          "longitude"  0.231372 ######
    4. "housing_median_age"  0.124641 #
    5.        "total_rooms"  0.101832 
    6.         "population"  0.095293 
    7.         "households"  0.089442 
    8.     "total_bedrooms"  0.089158 

Variable Importance: NUM_AS_ROOT:
    1.      "medi

In [100]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0)
